## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

D:\Anaconda3\envs\SAML11-32\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path_s = [path.curdir, "No_Security_Flash-S"]
project_path_s

['.', 'No_Security_Flash-S']

In [6]:
project_path_ns = [path.curdir, "No_Security_Flash-NS"]
project_path_ns

['.', 'No_Security_Flash-NS']

In [7]:
atprogram(path.abspath(path.join(*project_path_ns)), verbose=2)
atprogram(path.abspath(path.join(*project_path_s)), verbose=2, erase=False)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\ARM\CMSIS\5.4.0\CMSIS\Core\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/nvmctrl" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../../No_Security_Flash-S/trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.109\include" -I"../../../../shared"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: No_Security_Flash-NS.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\too

0

## Data Logging

In [8]:
live_plot = False

Create a figure for the plot.

In [9]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [10]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_no_security_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment No Security",
}

Stop criteria to pass to the logger:

In [11]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [12]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

# Store Data

In [13]:
import pickle
pickle.dump(data, open("no_security_flash_logger_data.p", "wb"))

# Load Data

In [14]:
data = pickle.load(open("no_security_flash_logger_data.p", "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [15]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [16]:
flash_write_charge, flash_write_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [17]:
flash_read_charge, flash_read_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [18]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(flash_write_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(flash_write_charge) - 1}")

MAX_NUM_BYTES: 6000


In [19]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [32]:
results = []
for y in [flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
    result = mod.fit(y[16:-16], pars, x=num_bytes[16:-16])
    print(result.fit_report())
    fig, grid = result.plot()
#     xlabel=f"{labels[result_type]['x']} " +
#            f"[{labels[result_type]['x_unit']}]",
#     ylabel=f"{labels[result_type]['y']} " +
#            f"[{labels[result_type]['y_unit']}]")
#     fig.canvas.set_window_title(
#         f"Residuals of {parameter_name}")
    fig.tight_layout()
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5968
    # variables        = 2
    chi-square         = 1.7290e-08
    reduced chi-square = 2.8981e-12
    Akaike info crit   = -158549.630
    Bayesian info crit = -158536.242
[[Variables]]
    slope:      2.6862e-08 +/- 1.2791e-11 (0.05%) (init = 0)
    intercept:  4.8416e-06 +/- 4.4256e-08 (0.91%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.867

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5968
    # variables        = 2
    chi-square         = 8.5246e-13
    reduced chi-square = 1.4289e-16
    Akaike info crit   = -217737.406
    Bayesian info crit = -217724.018
[[Variables]]
    slope:      8.4067e-10 +/- 8.9814e-14 (0.01%) (init = 0)
    intercept: -4.9010e-09 +/- 3.1075e-10 (6.34%) (init = 1)
[[Correlations]] (unrep

In [33]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title("Analysis No Security Flash")

In [34]:
charge_color = 'r'
time_color = 'b'

In [35]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [36]:
lines = []
lines += ax1.plot(num_bytes, flash_write_charge, charge_color+'-', label='Flash Write Charge')
lines += ax1.plot(num_bytes, flash_read_charge, charge_color+'--', label='Flash Read Charge')
lines += ax2.plot(num_bytes, flash_write_time, time_color+'-', label='Flash Write Time')
lines += ax2.plot(num_bytes, flash_read_time, time_color+'--', label='Flash Read Time')
ax1.legend(handles=lines)
# [flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
ax1.set_title(f"Flash Write Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} C per byte\n" +
             f"Flash Read Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} C per byte\n" +
             f"Flash Write Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"Flash Read Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [37]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(flash_write_charge, open("no_security_flash_write_charge.p", "wb"))
pickle.dump(flash_read_charge, open("no_security_flash_read_charge.p", "wb"))
pickle.dump(flash_write_time, open("no_security_flash_write_time.p", "wb"))
pickle.dump(flash_read_time, open("no_security_flash_read_time.p", "wb"))

## Write config file

In [26]:
import json

config = {}
config["name"] = "No Security Flash"
config["project_paths"] = [project_path_ns, project_path_s]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Flash Write"], 3: ["Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["Flash Write"],
                                        "load": ["Flash Read"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": 1}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)